# Project 2

## Molecular Data

The QM7 dataset consists of 7165 organic molecules, each of which is composed of up to 23 atoms.


Analysis will include:
1. Visualization
2. Representation
3. Ridge Regression
4.
5.

In [1]:
%matplotlib inline
import scipy
import numpy as np
import torch
import torchvision,torchvision.transforms
import sqlite3
import matplotlib
from matplotlib import pyplot as plt

data = scipy.io.loadmat('qm7.mat')

 The 3d coordinates of each atom in each molecule are available in the variable R. It is an array of size 7165×23×3
containing for each molecule and atom a triplet representing the 3d coordinates.

In [7]:
data['R'][0,0,:] # eg coordinates of first atom in first molecule

array([ 1.886438  , -0.00464873, -0.00823921], dtype=float32)

The variable Z is an array
of size 7165×23 which gives for each molecule and atom of the molecule the corresponding atomic number.
An atomic number of 1 corresponds to a hydrogen atom (H), the number 6 corresponds to carbon (C), the
numbers 7 and 8 to nitrogen (N) and oxygen (O) respectively, and finally, the number 16 corresponds to
sulfur (S). If the number is zero, then it indicates that there is no atom at this index, and the corresponding
3d coordinate should therefore be ignored.

In [8]:
data['Z'][0,0] # first atom of first molecule is carbon

6.0

 This allows for representing in the same array molecules of
different sizes. In addition to these geometrical features of the molecule, the dataset also provides for
each molecule its atomization energy (computed via quantum-chemical simulation). These atomization
energy values are stored in the variable T, an array of size 7165.

In [12]:
data['T'][0] # atomization energy values of first molecule

array([ -417.96,  -712.42,  -564.21, ..., -1662.1 , -1782.01, -1919.  ],
      dtype=float32)